In [16]:

import torch
from torch import nn
from torch.nn import Conv2d,MaxPool2d,Flatten,Linear
import torchvision
from torch.utils.data import DataLoader

dataset =torchvision.datasets.CIFAR10("../data", train=False, 
                                      transform=torchvision.transforms.ToTensor(),
                                      download=True)
dataloader = DataLoader(dataset, batch_size=1)

Files already downloaded and verified


In [14]:


class seqseq(nn.Module):
    def __init__(self):
        super(seqseq, self).__init__()
        # self.conv1 = Conv2d(3, 32, 5,padding=2)  # dilation 默认值是1, 即不进行空洞卷积
        # self.maxpool1 = MaxPool2d(2)
        # self.conv2 = Conv2d(32,32,5,padding=2)
        # self.maxpool2 = MaxPool2d(2)
        # self.conv3 = Conv2d(32,64,5,padding=2)
        # self.maxpool3 = MaxPool2d(2)
        # self.flatten = Flatten()
        # self.linear1 = Linear(1024,64)
        # self.linear2 = Linear(64,10)
        
        self.model1 = nn.Sequential(
            Conv2d(3, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )
        
        
    def forward(self, x):
        # x = self.conv1(x)
        # x = self.maxpool1(x)
        # x = self.conv2(x)
        # x = self.maxpool2(x)
        # x = self.conv3(x)
        # x = self.maxpool3(x)
        # x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.linear2(x)
        
        x = self.model1(x)
        
        return x
    


        


In [15]:
bunn = seqseq()
print(bunn)


seqseq(
  (model1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [19]:
loss_cross = nn.CrossEntropyLoss()
optim = torch.optim.SGD(bunn.parameters(),lr=0.01)

for epoch in range(20):
    running_loss = 0.0
    for data in dataloader:
        imgs, targets = data
        outputs = bunn(imgs)
        # print(outputs)
        # print(targets)
        result_loss = loss_cross(outputs, targets)
        # print(result_loss)
        optim.zero_grad()  # 把网络中之前的梯度清零
        result_loss.backward()  # 载入新的梯度
        optim.step()  # 根据新的梯度更新权重
        # print(result_loss)
        running_loss = running_loss + result_loss   # 单个epoch的loss总和
    print(running_loss.item()) # item()作用是把tensor值转换为数字
    

tensor(16157.4424, grad_fn=<AddBackward0>)
tensor(15406.6641, grad_fn=<AddBackward0>)
tensor(16027.9688, grad_fn=<AddBackward0>)
tensor(17792.8516, grad_fn=<AddBackward0>)
tensor(19788.5957, grad_fn=<AddBackward0>)
tensor(22079.9609, grad_fn=<AddBackward0>)
tensor(23882.6152, grad_fn=<AddBackward0>)
tensor(24432.3203, grad_fn=<AddBackward0>)
tensor(25283.4043, grad_fn=<AddBackward0>)
tensor(25742.2852, grad_fn=<AddBackward0>)
tensor(26111.8672, grad_fn=<AddBackward0>)
tensor(26727.9199, grad_fn=<AddBackward0>)
tensor(28205.5234, grad_fn=<AddBackward0>)
tensor(28667.1309, grad_fn=<AddBackward0>)
tensor(30291.7793, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)
tensor(nan, grad_fn=<AddBackward0>)


In [ ]:
input = torch.ones((64,3,32,32))


output = bunn(input)
print(output.shape)  # torch.size([64,10])  64是batchsize，10是10维


In [ ]:
# learning tensorboard
from torch.utils.tensorboard import SummaryWriter 
writer = SummaryWriter("../logs_bunn")
writer.add_graph(bunn,input)
writer.close()

In [ ]:
# learn the reshape
x = torch.tensor([0.1, 0.2, 0.3])
print(x.shape)
x
x = torch.reshape(x, (1, 3))
x

In [30]:
# leaning customize model

vgg16_false = torchvision.models.vgg16(weights=None)

# print(vgg16_false)
# vgg16_false.add_module("add_linear", nn.Linear(1000, 10))  # classifier外面加模块
# print(vgg16_false)

# vgg16_false.classifier.add_module("add_linear", nn.Linear(1000, 10))  # classifier里面加模块
# print(vgg16_false)

vgg16_false.classifier[6] = nn.Linear(4096,10)  # 直接内部修改
print(vgg16_false)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 网络保存


In [32]:
# 保存模型结构和参数
# 如果是自定义的模型，则在加载的时候，前面要加该模型的类
torch.save(vgg16_false, "../models/vgg16_false.pth")
# 仅保存参数，且保存成字典  
# 官方推荐
torch.sava(vgg16_false.state_dict(),"../models/vgg16_mothod2.pth")


## 模型加载

In [36]:
# 对方法一的加载
model = torch.load("../models/vgg16_false.pth")
print(model)

# 注意, 如果是自定义模型

class seqseq(nn.Module):
    def __init__(self):
        super(seqseq, self).__init__()
        # self.conv1 = Conv2d(3, 32, 5,padding=2)  # dilation 默认值是1, 即不进行空洞卷积
        # self.maxpool1 = MaxPool2d(2)
        # self.conv2 = Conv2d(32,32,5,padding=2)
        # self.maxpool2 = MaxPool2d(2)
        # self.conv3 = Conv2d(32,64,5,padding=2)
        # self.maxpool3 = MaxPool2d(2)
        # self.flatten = Flatten()
        # self.linear1 = Linear(1024,64)
        # self.linear2 = Linear(64,10)
        
        self.model1 = nn.Sequential(
            Conv2d(3, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )
        
        
    def forward(self, x):
        # x = self.conv1(x)
        # x = self.maxpool1(x)
        # x = self.conv2(x)
        # x = self.maxpool2(x)
        # x = self.conv3(x)
        # x = self.maxpool3(x)
        # x = self.flatten(x)
        # x = self.linear1(x)
        # x = self.linear2(x)
        
        x = self.model1(x)
        
        return x
    

model = torch.load("../models/bunn.pth")  # 前面不需要初始化模型
print(model)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:

# 对方法二的加载
vgg16_mothod2 = torchvision.models.vgg16(weights=None)
vgg16_mothod2.load_state_dict(torch.load("../models/vgg16_mothod2.pth"))
# model = torch.load("../models/vgg16_mothod2.pth")
print(model)
